In [2]:
import pandas as pd

In [10]:
user_card_matrix=pd.read_excel(r'./user_card_matrix_for_rec.xlsx',encoding='utf-16')

In [14]:
user_card_matrix=user_card_matrix.rename(columns={'Unnamed: 0':'user'})

In [16]:
user_card_matrix.head()

,user,三信銀行-VISA白金卡,上海商銀-TeresaCard,上海商銀-小小兵Bello回饋卡,上海商銀-簡單卡,土地銀行-JCB一卡通聯名晶緻卡,中國信託-ANA聯名卡,中國信託-LINE Pay卡,中國信託-MUJI無印良品聯名卡,中國信託-Yahoo聯名卡,...,臺灣銀行-導盲犬認同卡,遠東商銀-Bankee信用卡,遠東商銀-快樂信用卡,聯邦銀行-全國加油聯名卡,聯邦銀行-幸福御守御璽卡,聯邦銀行-法拉利無限卡,聯邦銀行-紅利卡,聯邦銀行-理財型白金卡,聯邦銀行-微風聯名卡,聯邦銀行-賴點一卡通御璽卡
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
colums_list=list(user_card_matrix.columns.values)[1:]

In [27]:
spark_als_data=pd.melt(user_card_matrix, id_vars=['user'], value_vars=colums_list)

In [28]:
len(spark_als_data)

476820

In [ ]:
spark_als_data.to_excel(r'./spark_als_data.xlsx',encoding='')